In [1]:
import numpy as np
import subprocess
from scipy import signal

In [2]:
epsilon=0.01
num_iteration=10

In [3]:
def check(c,c1):
    for num1,num2 in zip(c,c1):
        if(np.abs(num1-num2)>epsilon):
            print "Not working"
            print str(num1)+" "+str(num2)+" "+str(np.abs(num1-num2))

In [4]:
#testing convolution with multiplication
subprocess.call(["make"])
for i in range(num_iteration):
    size1=np.random.randint(10,100)
    size2=size1//np.random.randint(1,10)
    padding=np.random.randint(0,20)
    a=np.random.randn(size1,size1).astype(np.float32)
    b=np.random.randn(size2,size2).astype(np.float32)
    a_columMajor=a.T
    b_columnMajor=b.T
    f = open("input1.txt", "w")
    for elem in a_columMajor:
        f.write(" ".join(str(num) for num in elem))
        f.write(" ")
    f.close();
    f=open("input2.txt","w")
    for elem in b_columnMajor:
        f.write(" ".join(str(num) for num in elem))
        f.write(" ")
    f.close()
    subprocess.call(["./main","convolution_mult",str(padding),"input1.txt",str(size1),"input2.txt",
                     str(size2),"output.txt"])
    f=open("output.txt","r")
    line=f.readline()
    c=np.array([np.float32(x) for x in line.split()])
    f.close()
    a1=np.pad(a, (padding,padding), 'constant', constant_values=(0, 0))
    c1=np.squeeze(signal.correlate2d(a1,b,mode="valid").T.reshape(-1,1))
    
    check(c,c1)

In [5]:
#testing activation
subprocess.call(["make"])
for i in range(num_iteration):
    size1=np.random.randint(1,100)
    size2=np.random.randint(1,100)
    a=np.random.randn(size1,size2).astype(np.float32)
    f = open("input.txt", "w")
    for elem in a.T:
        f.write(" ".join(str(num) for num in elem))
        f.write(" ")
    f.close();
    option=np.random.randint(1,3)
    if(option==1):
        c1=np.squeeze(np.maximum(a,0).T.reshape(-1,1))
        subprocess.call(["./main","activation","relu","input.txt",str(size1),str(size2),"output.txt"])
        f=open("output.txt","r")
        line=f.readline()
        c=np.array([np.float32(x) for x in line.split()])
        f.close()
        check(c,c1)
    else:
        c1=np.squeeze(np.tanh(a).T.reshape(-1,1))
        subprocess.call(["./main","activation","tanh","input.txt",str(size1),str(size2),"output.txt"])
        f=open("output.txt","r")
        line=f.readline()
        c=np.array([np.float32(x) for x in line.split()])
        f.close()
        check(c,c1)